<a href="https://colab.research.google.com/github/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/blob/main/Sentiment_Net_dualis_extend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
#emoji_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5e7d7d29f58be438f33fa0c0bc2fd251dea0b453/emoji.py"
#!rm emoji.py
#!wget $emoji_url
#import emoji

In [3]:
! pip install emoji
import emoji

     |████████████████████████████████| 168 kB 5.3 MB/s 
  Created wheel for emoji: filename=emoji-1.6.0-py3-none-any.whl size=168256 sha256=2f65cba8b6f6fab07a88889b2cd00c4624d7407503c44937ffb05a348140858e
  Stored in directory: /root/.cache/pip/wheels/f7/d7/74/c720aaf345a042b0c2d74361873258c5e8649b7f11b2ccce49
Successfully built emoji


In [4]:

emoji.demojize("hello 👍")

'hello :thumbs_up:'

##Szószedet

In [5]:
!rm SampleSubmission*
!rm TEST*
!rm TRAIN*
!rm *zip*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip
df=pd.read_csv("TEST.csv")

rm: cannot remove 'SampleSubmission*': No such file or directory
rm: cannot remove 'TEST*': No such file or directory
rm: cannot remove 'TRAIN*': No such file or directory
rm: cannot remove '*zip*': No such file or directory
--2021-10-11 17:08:09--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip [following]
--2021-10-11 17:08:09--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving raw.githubus

In [6]:
df.head()
alltext1=df.text
print (len(alltext1))

2640


In [7]:
df2=pd.read_csv("TRAIN.csv")
alltext2=df2.text
print (len(alltext2))

12000


In [8]:
alltext1=list(alltext1)
alltext2=list(alltext2)
alltext=alltext1+alltext2
print(f" {len(alltext)}")


 14640


In [9]:
!pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()
print(spell.correction("@frustrating"))

     |████████████████████████████████| 2.7 MB 5.4 MB/s 
frustrating


In [10]:
import re

def remove_URL(text):
    """Remove URLs from a text string"""
    return re.sub(r"http\S+", "", text)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def is_number(inp:str):
    a=re.search("[\d]",inp)
    if a==None: 
        return(False)
    return(True)

def strip_all_entities(text):
    import emoji
    import re,string
    entity_prefixes = ['“','@','#',"!","?",":","_"]
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in ['“','@','#',"!","?"]:
                words.append(word)

    return ' '.join(words)

def de_ex(txt):
    txt=txt.lower()
    exwords={"it's":"it is",
    "aren't":"are not",
    "i've":"i have",
    "can't":"can not",
    "didn't":"did not",
    "i'm":"i am",
    "that's":"that is",
    "let's":" let is",
    "she's":"she is",
    "we're":"we are",
    "they've":"they have",
    "fleet's":"fleet is ",
    "there's":"there is",
    "i'll": "i will"
    }
    txt_list=txt.split()
    for w in txt_list:
        if w in exwords:
            txt=txt.replace(w,exwords[w])
    return (txt)



def query_sentence(index):
    print(f"orig: {alltext[index]}")
    spell=SpellChecker()
    inp=alltext[index]
    inp=remove_URL(inp)
    inp=emoji.demojize(inp)
    inp=de_ex(inp)
    print("demojize: ",inp)
    inp=strip_all_entities(inp)
    print("STRIP: ",inp)
    txt=inp.replace(","," ").replace("."," ").replace("!"," _exclamation_ ").replace("#"," ").replace("???","?").replace("??","?").replace("?"," _questionmark_ ").replace("  "," ").replace("  "," ").replace('"',"").replace("“","")
    print("after replace=",txt)
    #print(txt)
    normal=""
    for i in txt.split(" "):
        w1=i.lower().strip()
        if "_" in w1 or ":" in w1:
            w2=w1
        else:
            if w1 !="":
                w2=spell.correction(w1).strip()
            else:
                w2=""
            if w2=="i":
                pass
                #print(f"chek: {w1}, {w2}")
            if is_number(w2):
                w2=""    
        normal += w2+" "
    normal=normal.replace("::",": :")
    print(f"  conv: {normal}")
    return(normal)

In [11]:
df2.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570018445695148033,negative,1.0,Customer Service Issue,0.656,United,NaN,tpensari,NaN,0,@united your customer service is terrible! Sto...,NaN,2015-02-23 16:32:42 -0800,NaN,NaN


# az ötlet: A teszt adatbázisból ki lehet szűrni a negatív eseteket.
### csak a szöveg kell az eredeti adatbázisból
### a test adatokat negatívra vesszük.

Persze ha továbbgondoljuk, akkor lehetne egy (positive, neutral) selectort tanítani. Hiszen a negatív kommentekről mindent tudunk.
Ezeket könnyü is kiválogatni a rendszerből, kérdés hogy rakjuk majd össze. 
Ezt majd megoldjuk.
A legfontosabb, hogy tanulna egy ilyen "duális kimenetre a rendszerünk"


In [12]:
def listing(dfx,printing=False):
    t_selector=[]
    count_neg=0
    count_pos=0
    for i in range(len(dfx)):
    
        reason=dfx.loc[i].negativereason
        airline_sentiment="NONE"
        if "airline_sentiment" in dfx.columns:
            airline_sentiment=dfx.loc[i].airline_sentiment
            
        txt=dfx.loc[i].text
        cond=[reason=="Flight Booking Problems", 
                    reason=="Bad Flight", 
                    reason=="Late Flight", 
                    reason=="Lost Luggage", 
                    reason=="Customer Service Issue",
                    reason=="Cancelled Flight",
                    reason=="Flight Attendant Complaints",
                    reason=="longlines",
                    reason=="Damaged Luggage",
                    reason=="Can't Tell"
                    ]
        if any(cond):
            count_neg+=1
            t_selector.append(True) # kell ez az adat nekünk
            if printing:
                print(f"negative:{count_neg} --  {reason}  -- {airline_sentiment} -- {txt}" )
        else:
            count_pos+=1
            if printing:
                print(f"??:{count_pos} --  {reason}  -- {airline_sentiment} -- {txt}" )
            t_selector.append(False) # ez nem kell nem negatív
            
    print(f"Summa: negative: {count_neg}   --  positive: {count_pos}")
    return(t_selector)

In [13]:
print (len(df2))

12000


In [14]:
selector=listing(df)

Summa: negative: 1661   --  positive: 979


In [15]:
print (len(selector))

2640


In [16]:
df_negativ_test=df[selector]

##Mentés

In [17]:
t=[]
for i in range(len(alltext)):
    print(f"{i}", end=" ")
    t.append(query_sentence(i))


0 orig: @united well someone should tell that to the employees at the Denver baggage claim. Still no bag!!!!
demojize:  @united well someone should tell that to the employees at the denver baggage claim. still no bag!!!!
STRIP:  well someone should tell that to the employees at the denver baggage claim still no bag!!!!
after replace= well someone should tell that to the employees at the denver baggage claim still no bag _exclamation_ _exclamation_ _exclamation_ _exclamation_ 
  conv: well someone should tell that to the employees at the denver baggage claim still no bag _exclamation_ _exclamation_ _exclamation_ _exclamation_  
1 orig: @AmericanAir Let's all have a extraordinary week and make it a year to remember #GoingForGreat 2015 thanks so much American Airlines!!!
demojize:  @americanair  let is all have a extraordinary week and make it a year to remember #goingforgreat 2015 thanks so much american airlines!!!
STRIP:  let is all have a extraordinary week and make it a year to remem

KeyboardInterrupt: ignored

In [ ]:
s=set()
for sent in t:
    wlist=sent.strip().split()
    for w in wlist:
        s.add(w)
print (len(s))
ordered=list(s)
ordered.sort()

In [ ]:
print(ordered)

In [ ]:
sentence=pd.DataFrame(t)
sentence.columns=["Sentence"]
sentence.to_csv("Sentences_emoji.csv")

In [ ]:
wordlist=pd.DataFrame(ordered)
wordlist.columns=["Words"]
wordlist.to_csv("words_emoji.csv")

###---------------------------------


###Adatbetöltés SZAVAK BETÖLTÉSE

In [164]:
import pandas as pd

In [165]:
#words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv"
words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/words_emoji_2.csv"
words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/words_emoji_3.csv"


In [166]:
!rm words* 
!wget $words_url

--2021-10-11 19:29:53--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/words_emoji_3.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5de86fcaa648f3b456cfcc6514b6a045ca095c17/words_emoji_3.csv [following]
--2021-10-11 19:29:53--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5de86fcaa648f3b456cfcc6514b6a045ca095c17/words_emoji_3.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123520 (121K) [text/plain]
Saving to: ‘words_emoji_3.csv’

words_emoj

In [167]:
words=pd.read_csv("words_emoji_3.csv",index_col=0)
words.head()
words_list=list(words["Words"])

In [168]:
word_dict={v:i+1 for i,v in enumerate(words_list)}

### adatbetöltés mondatok betöltése

In [169]:
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/Sentences.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/b9567d54968bba4d03ce8c3cac5ecdc4fe1c6031/Sentences_emoji.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/Sentences_emoji_2.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/Sentences_emoji_3.csv"


In [170]:
!rm Sentences*
!wget $sentences_url
sentences=pd.read_csv("Sentences_emoji_3.csv",index_col=0)
sentences.head()
sentences_list=list(sentences["Sentence"])

--2021-10-11 19:29:54--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/Sentences_emoji_3.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5de86fcaa648f3b456cfcc6514b6a045ca095c17/Sentences_emoji_3.csv [following]
--2021-10-11 19:29:54--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5de86fcaa648f3b456cfcc6514b6a045ca095c17/Sentences_emoji_3.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1470690 (1.4M) [text/plain]
Saving to: ‘Sentences_emoji_3

##Tanulás

In [171]:
!rm *zip*
!rm TEST*
!rm TRAIN*
!rm SampleSubmission*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip


--2021-10-11 19:29:55--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip [following]
--2021-10-11 19:29:55--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

In [172]:
df=pd.read_csv("TRAIN.csv")

In [173]:
df.head(4)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570018445695148033,negative,1.0,Customer Service Issue,0.6560,United,NaN,tpensari,NaN,0,@united your customer service is terrible! Sto...,NaN,2015-02-23 16:32:42 -0800,NaN,NaN
1,568830255282040832,neutral,1.0,NaN,NaN,Southwest,NaN,leavenodoubt131,NaN,0,@SouthwestAir I will dm you now,NaN,2015-02-20 09:51:15 -0800,NaN,Tijuana
2,569296356143050754,negative,1.0,longlines,0.3462,United,NaN,ScottyNeuman,NaN,1,@united congratulations united. Fail again. Sk...,"[39.85760174, -104.66669625]",2015-02-21 16:43:22 -0800,"Montana, God's country\n",Pacific Time (US & Canada)
3,569936640669581313,positive,1.0,NaN,NaN,United,NaN,koploperfan1992,NaN,0,@united look at this beauty 😉 dc 10 united air...,"[51.322819, 5.3576218]",2015-02-23 11:07:38 -0800,Winterfell,NaN


In [174]:
for i in range(len(df)):
    if "😊" in df.loc[i].text:
        print (f"{i:6}. {df.loc[i].airline_sentiment:9} -- {df.loc[i].retweet_count:5} --  {df.loc[i].text}")

     8. positive  --     0 --  @SouthwestAir a nice trip back home after a looong vaca 😊🌴 http://t.co/9TRvfnCedL
   752. positive  --     0 --  @united Thank you!! 😊
  1447. neutral   --     0 --  @JetBlue "Do what you love and you'll never work a day in your life." 😊
  1494. positive  --     0 --  @SouthwestAir Great job with the Passbook integration! It’ll really help streamline the commute to San Fran! 😊☕📲✈
  1781. positive  --     0 --  @united EWR agent Barbara was FABULOUS and an example of CUST. SERV. A pleasure talking to you😊 http://t.co/KMQuLY9g5E
  2740. positive  --     1 --  @JetBlue @Maddie_Flood Your airline sounds outstanding and your Twitter feed is clearly extremely useful. Keep up the great work 😊
  2914. positive  --     0 --  @USAirways the entire flight crew on flight 738 from BOS to PHL is doing a wonderful job and making this experience not totally suck. 😊
  3109. positive  --     0 --  @JetBlue be flying soon to NYC on your airline, of all airlines I've flown y

In [175]:
for i in range(len(df)):
    if df.loc[i].retweet_count>0:
        print (f"{i:6}. {df.loc[i].airline_sentiment:9} -- {df.loc[i].retweet_count:5} --  {df.loc[i].text}")

     2. negative  --     1 --  @united congratulations united. Fail again. Sky at a gate with a plane full of people waiting on 2. Weather gets worse now we can't leave.
    10. neutral   --     3 --  @southwestair has 22 employees on Social Care Team, 4 staff the Listening Ctr. at any time-rest of spots filled by other areas. #RaganDisney
    66. negative  --     1 --  @united that may be true however after 4 weeks matter is still unresolved and I can not get through to a supervisor or manager to help
   112. negative  --     1 --  @united  very disappointed by the service starting from gate operator at BOS , who was  rude in stopping a carry-on bag...
   118. negative  --     1 --  @united what a pointless tweet. At least @AmericanAir asked me to follow them to try and resolve
   148. negative  --     1 --  @united how much longer United?? Been on the phone for over an hour to straighten out a star alliance upgrade!!! http://t.co/U1ViEuiDF5
   153. positive  --     4 --  It really is

In [176]:
sentences_list[10420]

'you re the reason this whole travel experience has been a nightmare '

In [177]:
df.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570018445695148033,negative,1.0,Customer Service Issue,0.656,United,NaN,tpensari,NaN,0,@united your customer service is terrible! Sto...,NaN,2015-02-23 16:32:42 -0800,NaN,NaN


In [178]:
df["clean_text"]=sentences_list[2640:]

In [235]:
df.head(10)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentence_code
0,570018445695148033,negative,1.0000,Customer Service Issue,0.6560,United,NaN,tpensari,NaN,0,@united your customer service is terrible! Sto...,NaN,2015-02-23 16:32:42 -0800,NaN,NaN,your customer service is terrible _exclamation...,"[9595, 2343, 7608, 4613, 8497, 364, 8137, 4474..."
1,568830255282040832,neutral,1.0000,NaN,NaN,Southwest,NaN,leavenodoubt131,NaN,0,@SouthwestAir I will dm you now,NaN,2015-02-20 09:51:15 -0800,NaN,Tijuana,i will do you now,"[4273, 9420, 2763, 9586, 5846]"
2,569296356143050754,negative,1.0000,longlines,0.3462,United,NaN,ScottyNeuman,NaN,1,@united congratulations united. Fail again. Sk...,"[39.85760174, -104.66669625]",2015-02-21 16:43:22 -0800,"Montana, God's country\n",Pacific Time (US & Canada),congratulations united fail again sky at a gat...,"[2085, 8976, 3298, 534, 7798, 929, 366, 3707, ..."
3,569936640669581313,positive,1.0000,NaN,NaN,United,NaN,koploperfan1992,NaN,0,@united look at this beauty 😉 dc 10 united air...,"[51.322819, 5.3576218]",2015-02-23 11:07:38 -0800,Winterfell,NaN,look at this beauty :winking_face: do united ...,"[5134, 929, 8560, 1140, 357, 2763, 8976, 585, ..."
4,569542242983124992,positive,1.0000,NaN,NaN,Delta,NaN,sandyy889,NaN,0,@JetBlue Landing! As usual great flight wiyh a...,NaN,2015-02-22 09:00:26 -0800,NaN,America/Detroit,landing _exclamation_ as usual great flight wi...,"[4873, 364, 877, 9092, 3859, 3484, 9464, 366, ..."
5,567738192426590209,neutral,0.6631,NaN,0.0000,United,NaN,rajuchinthala,NaN,0,Thx! Stand by! “@united: @rajuchinthala I know...,NaN,2015-02-17 09:31:47 -0800,Indianapolis,Eastern Time (US & Canada),the _exclamation_ stand by _exclamation_ i kno...,"[8529, 364, 8059, 1528, 364, 4273, 4822, 4633,..."
6,568558135637463040,positive,0.6634,NaN,NaN,United,NaN,TonyLauro,NaN,0,"@united Fair enough. I don't usually rant, but...",NaN,2015-02-19 15:49:57 -0800,"Dallas, TX",Central Time (US & Canada),fair enough i don i usually rant but it is goo...,"[3305, 3061, 4273, 2786, 4273, 9093, 6806, 151..."
7,569693713087082497,negative,0.6666,Can't Tell,0.3355,American,NaN,jkhoey,NaN,0,@AmericanAir 37 minutes??????,NaN,2015-02-22 19:02:20 -0800,"New York, NY",Eastern Time (US & Canada),minutes _questionmark_,"[5489, 365]"
8,568591059602829314,positive,1.0000,NaN,NaN,Southwest,NaN,iknowmatt2,NaN,0,@SouthwestAir a nice trip back home after a lo...,NaN,2015-02-19 18:00:47 -0800,~~ Rhode Island ~~,Eastern Time (US & Canada),a nice trip back home after a looong vaca :smi...,"[366, 5747, 8806, 1037, 4150, 531, 366, 5140, ..."
9,569493640990625793,negative,1.0000,Late Flight,1.0000,United,NaN,_Charette_,NaN,0,@united flight 403 is delayed 40 min bc a miss...,NaN,2015-02-22 05:47:19 -0800,SE AZ By Way of HV,Mountain Time (US & Canada),flight is delayed min be a missing screw and...,"[3484, 4613, 2481, 5473, 1125, 366, 5518, 7504..."


In [180]:
df_pos_neutral=df[(df["airline_sentiment"]=="positive") | (df["airline_sentiment"]=="neutral")]

In [181]:
df_pos_neutral.head(4)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text
1,568830255282040832,neutral,1.0000,NaN,NaN,Southwest,NaN,leavenodoubt131,NaN,0,@SouthwestAir I will dm you now,NaN,2015-02-20 09:51:15 -0800,NaN,Tijuana,i will do you now
3,569936640669581313,positive,1.0000,NaN,NaN,United,NaN,koploperfan1992,NaN,0,@united look at this beauty 😉 dc 10 united air...,"[51.322819, 5.3576218]",2015-02-23 11:07:38 -0800,Winterfell,NaN,look at this beauty :winking_face: do united ...
4,569542242983124992,positive,1.0000,NaN,NaN,Delta,NaN,sandyy889,NaN,0,@JetBlue Landing! As usual great flight wiyh a...,NaN,2015-02-22 09:00:26 -0800,NaN,America/Detroit,landing _exclamation_ as usual great flight wi...
5,567738192426590209,neutral,0.6631,NaN,0.0,United,NaN,rajuchinthala,NaN,0,Thx! Stand by! “@united: @rajuchinthala I know...,NaN,2015-02-17 09:31:47 -0800,Indianapolis,Eastern Time (US & Canada),the _exclamation_ stand by _exclamation_ i kno...


In [182]:
def print_df(df,start=0,stop=0):
    for i in range(start,stop):
        print(f"{i:4}.  --{df.loc[i].text} -- {df.loc[i].clean_text}")

In [183]:
print_df(df,1,10)

   1.  --@SouthwestAir  I will dm you now -- i will do you now 
   2.  --@united congratulations united. Fail again. Sky at a gate with a plane full of people waiting on 2. Weather gets worse now we can't leave. -- congratulations united fail again sky at a gate with a plane full of people waiting on  weather gets worse now we can not leave 
   3.  --@united look at this beauty 😉 dc 10 united airlines 😉 http://t.co/MvYoizRPdE -- look at this beauty :winking_face: do  united airlines :winking_face: 
   4.  --@JetBlue Landing! As usual great flight wiyh a great crew. Hello sunny West Palm Beach ! #jetbluerocks -- landing _exclamation_ as usual great flight wiyh a great crew hello sunny west palm beach 
   5.  --Thx! Stand by! “@united: @rajuchinthala I know it's frustrating and I do appreciate your patience while we try to get you on your way. ^JH” -- the _exclamation_ stand by _exclamation_ i know it is frustrating and i do appreciate your patience while we try to get you on your way th

In [184]:
def create_all_sentence(df):
    all_sent=[]
    for sentence in df.clean_text:
        sent_list=sentence.strip().split()
        out=[]
        for word in sent_list:
            out.append(word_dict[word]) 
        all_sent.append(out)
    return(all_sent)


In [185]:
all_sent=create_all_sentence(df)
df["sentence_code"]=all_sent

In [186]:

all_sent=create_all_sentence(df_pos_neutral)

In [187]:
df_pos_neutral["sentence_code"]=all_sent

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [188]:
df_pos_neutral.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentence_code
1,568830255282040832,neutral,1.0,NaN,NaN,Southwest,NaN,leavenodoubt131,NaN,0,@SouthwestAir I will dm you now,NaN,2015-02-20 09:51:15 -0800,NaN,Tijuana,i will do you now,"[4273, 9420, 2763, 9586, 5846]"


In [189]:
table=list(df["sentence_code"]) # typo
lens=max([len(x) for x in table])
sentiment=list(df_pos_neutral["airline_sentiment"])


In [190]:
def ylabel_convert(sentiment):
    ytrain=[]
    for sent in sentiment:
        if sent=="negative":
            o=[1,0]
        if sent=="neutral":
            o=[0.5,0.5]
        if sent=="positive":
            o=[0,1]
        ytrain.append(o)
    return(ytrain)

def ylabel_convert3(sentiment):
    ytrain=[]
    for sent in sentiment:
        if sent=="negative":
            o=[1,0,0]
        if sent=="neutral":
            o=[0,1,0]
        if sent=="positive":
            o=[0,0,1]
        ytrain.append(o)
    return(ytrain)

def ylabel_convert2(sentiment):
    ytrain=[]
    for sent in sentiment:
      
        if sent=="neutral":
            o=[1,0]
        if sent=="positive":
            o=[0,1]
        ytrain.append(o)
    return(ytrain)


        

In [191]:
ytrain=ylabel_convert2(sentiment)

In [192]:
x0=[0 for _ in range(lens) ]
xtrain=[]
for sent in df_pos_neutral["sentence_code"]:
    o1=list(x0[0:lens-len(sent)]+list(sent))
    xtrain.append(o1)

In [193]:
x_train=xtrain[0:-20]
y_train=ytrain[0:-20]
x_test=xtrain[-20:]
y_test=ytrain[-20:]

In [194]:
len(x_train)

4463

In [195]:


lstm_size=lens
max_input_length=lens
embedding_size=10 #(100: 73%)
n_words=len(table)
n_out=len(y_train[0])

In [196]:
lens
n_out

2

In [197]:

# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed, Flatten, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy, mean_squared_error
from tensorflow.keras import backend as K

K.clear_session
x= Input(shape=(max_input_length,))
embedded_x=Embedding(n_words+1,embedding_size, input_length=max_input_length-1, mask_zero=True)(x)
lstm_output= LSTM(lstm_size,return_sequences=True)(embedded_x)
lstm_output= LSTM(lstm_size,return_sequences=True)(lstm_output)


lstm_output=Flatten()(lstm_output)
#Dense_out= Dense(33, activation="softmax")(lstm_output)

predictions= Dense(n_out, activation="softmax")(lstm_output)
model=Model(inputs=x, outputs=predictions,)



In [198]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 59)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 59, 10)            120010    
_________________________________________________________________
lstm_2 (LSTM)                (None, 59, 59)            16520     
_________________________________________________________________
lstm_3 (LSTM)                (None, 59, 59)            28084     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3481)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6964      
Total params: 171,578
Trainable params: 171,578
Non-trainable params: 0
_____________________________________________________

In [199]:
import numpy as np

In [200]:
#train_x = np.asarray(xtrain)
#train_y = np.asarray(ytrain)

In [201]:
# Loss 

loss = mean_squared_error #categorical_crossentropy # One-hot enkódolt kimenetünk van. Mit is használunk?

# Optimizer
optimizer = Adam() #Ízlés szerint...
 
# Compilation
#############

model.compile(optimizer=optimizer,loss=loss)

In [203]:
# Illesszük az adatra a modellt.
history=model.fit(x=xtrain,y=ytrain,validation_data=( x_test,y_test), epochs=6, batch_size=2,)


Epoch 1/6
1683/2242 [=====================>........] - ETA: 4:43 - loss: 0.0840

KeyboardInterrupt: ignored

In [204]:
pred=model.predict(x_train)

In [205]:
for i in range(len(pred)):
    print(f"{i}, {pred[i]}, {y_train[i]}")



0, [0.7761447  0.22385533], [1, 0]
1, [0.0714925  0.92850745], [0, 1]
2, [9.4285223e-04 9.9905711e-01], [0, 1]
3, [0.95536965 0.0446303 ], [1, 0]
4, [0.02005879 0.9799412 ], [0, 1]
5, [0.04196197 0.95803803], [0, 1]
6, [0.96487635 0.03512366], [1, 0]
7, [0.23447981 0.76552016], [0, 1]
8, [0.45795768 0.5420423 ], [1, 0]
9, [0.01877449 0.98122555], [0, 1]
10, [0.34465137 0.65534866], [1, 0]
11, [7.744820e-04 9.992255e-01], [0, 1]
12, [0.8684002  0.13159983], [1, 0]
13, [1.3523537e-04 9.9986470e-01], [0, 1]
14, [9.9947804e-01 5.2198971e-04], [1, 0]
15, [0.29041442 0.7095856 ], [0, 1]
16, [0.65388006 0.3461199 ], [0, 1]
17, [0.32723334 0.6727667 ], [1, 0]
18, [0.45027888 0.5497211 ], [0, 1]
19, [0.19304782 0.8069521 ], [0, 1]
20, [0.93906724 0.06093271], [1, 0]
21, [0.2409496 0.7590504], [0, 1]
22, [0.19028118 0.80971885], [1, 0]
23, [0.00861623 0.9913838 ], [0, 1]
24, [0.9509011  0.04909888], [1, 0]
25, [0.51759624 0.48240373], [1, 0]
26, [0.7661834  0.23381658], [1, 0]
27, [0.03236511 0.

### Model usage

In [206]:
test_df=pd.read_csv("TEST.csv")

In [207]:
print(len(test_df))

2640


In [208]:
test_df.head(130)

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570031016800161792,1.0000,Lost Luggage,1.0000,United,NaN,tpensari,NaN,0,@united well someone should tell that to the e...,NaN,2015-02-23 17:22:39 -0800,NaN,NaN
1,569948646717812736,1.0000,NaN,NaN,American,NaN,Metalmikefisher,NaN,0,@AmericanAir Let's all have a extraordinary we...,NaN,2015-02-23 11:55:21 -0800,NaN,Eastern Time (US & Canada)
2,569469326547222528,0.6811,Late Flight,0.6811,United,NaN,jlongoria21,NaN,0,@united since I have an international connecti...,NaN,2015-02-22 04:10:42 -0800,"Rio Grande Valley, Texas",Central Time (US & Canada)
3,569512239209824256,1.0000,Customer Service Issue,0.6812,Delta,NaN,cshells4,NaN,0,@JetBlue woulda been nice of you to let us kno...,NaN,2015-02-22 07:01:13 -0800,Connecticut,NaN
4,567842370533867520,1.0000,Customer Service Issue,1.0000,US Airways,NaN,suzanneboles,NaN,0,@USAirways contd.. They put her on 7 pm flite ...,NaN,2015-02-17 16:25:45 -0800,"London, ON'",Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,570285904809598977,1.0000,NaN,NaN,Virgin America,NaN,sjespers,NaN,0,@VirginAmerica Thanks!,NaN,2015-02-24 10:15:29 -0800,"San Francisco, CA",Pacific Time (US & Canada)
126,568650058780041216,1.0000,Customer Service Issue,1.0000,US Airways,NaN,lj_verde,NaN,0,@USAirways never flying #USAirways again. Horr...,NaN,2015-02-19 21:55:13 -0800,DC,Eastern Time (US & Canada)
127,568494200163074048,0.3796,NaN,0.0000,US Airways,NaN,J_0_Y,NaN,0,@USAirways I just think it's weird to have mil...,"[42.95119138, -87.90117857]",2015-02-19 11:35:54 -0800,"Washington, DC",NaN
128,570059530567962624,1.0000,Can't Tell,1.0000,United,NaN,_mhertz,NaN,1,@united @AmericanAir so that's it? It just end...,NaN,2015-02-23 19:15:57 -0800,NaN,Pacific Time (US & Canada)


In [209]:
test_df["clean_text"]=sentences_list[:2640]

In [210]:
def code_sentences(df):
    all_sent=[]
    for sentence in df.clean_text:
        sent_list=sentence.strip().split()
        out=[]
        for word in sent_list:
            out.append(word_dict[word]) 
        all_sent.append(out)
    df["sentence_code"]=all_sent
    return (df)

In [211]:
test_df=code_sentences(test_df)

In [212]:
def push_sentences(df,lens=33):
    table=list(df["sentence_code"]) # typo
    
    print(lens)
    x0=[0 for _ in range(lens) ]
    x=[]
    for sent in table:
        o1=list(x0[0:lens-len(sent)]+list(sent))
        x.append(o1)
    return(x)

In [213]:
test_df.head(1)

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentence_code
0,570031016800161792,1.0,Lost Luggage,1.0,United,NaN,tpensari,NaN,0,@united well someone should tell that to the e...,NaN,2015-02-23 17:22:39 -0800,NaN,NaN,well someone should tell that to the employees...,"[9345, 7916, 7696, 8473, 8524, 8646, 8529, 302..."


In [214]:

xx=push_sentences(test_df,lens=59)

59


In [215]:
test_out=model.predict(xx)

In [216]:
test_out

array([[0.1120205 , 0.88797945],
       [0.00148423, 0.9985158 ],
       [0.62064636, 0.37935358],
       ...,
       [0.62186617, 0.3781338 ],
       [0.00375329, 0.99624676],
       [0.996128  , 0.00387199]], dtype=float32)

In [217]:
def decode3(model_out):
    y=[]
    for mo in model_out:
        if mo[0]>mo[1] and mo[0]>mo[2]:
            o="negative"    
        if mo[1]>mo[0] and mo[1]>mo[2]:
            o="neutral"    
        if mo[2]>mo[0] and mo[2]>mo[1]:
            o="positive"    
        y.append(o)
    return(y)


  

In [218]:
def decode(model_out):
    y=[]
    for mo in model_out:
        dif=abs(mo[0]-mo[1])
        if dif < 0.2 :
            o="neutral"
        else:
            if mo[0]>mo[1] :
                o="negative"    
            else:
                o="positive"    
        y.append(o)
    return(y)



In [219]:
def decode2(model_out):
    y=[]
    for mo in model_out:
        if mo[0]>mo[1] :
            o="neutral"    
        else:
            o="positive"    
        y.append(o)
    return(y)


In [220]:
prediction=decode2(test_out)

In [221]:
print (len(prediction))
test_df.predicted=prediction

2640


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [222]:
test_df.head(5)

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentence_code
0,570031016800161792,1.0000,Lost Luggage,1.0000,United,NaN,tpensari,NaN,0,@united well someone should tell that to the e...,NaN,2015-02-23 17:22:39 -0800,NaN,NaN,well someone should tell that to the employees...,"[9345, 7916, 7696, 8473, 8524, 8646, 8529, 302..."
1,569948646717812736,1.0000,NaN,NaN,American,NaN,Metalmikefisher,NaN,0,@AmericanAir Let's all have a extraordinary we...,NaN,2015-02-23 11:55:21 -0800,NaN,Eastern Time (US & Canada),let is all have a extraordinary week and make ...,"[4973, 4613, 627, 4010, 366, 3272, 9327, 703, ..."
2,569469326547222528,0.6811,Late Flight,0.6811,United,NaN,jlongoria21,NaN,0,@united since I have an international connecti...,NaN,2015-02-22 04:10:42 -0800,"Rio Grande Valley, Texas",Central Time (US & Canada),since i have an international connection at am...,"[7753, 4273, 4010, 696, 4554, 2092, 929, 663, ..."
3,569512239209824256,1.0000,Customer Service Issue,0.6812,Delta,NaN,cshells4,NaN,0,@JetBlue woulda been nice of you to let us kno...,NaN,2015-02-22 07:01:13 -0800,Connecticut,NaN,woulda been nice of you to let us know that or...,"[9520, 1151, 5747, 5899, 9586, 8646, 4973, 907..."
4,567842370533867520,1.0000,Customer Service Issue,1.0000,US Airways,NaN,suzanneboles,NaN,0,@USAirways contd.. They put her on 7 pm flite ...,NaN,2015-02-17 16:25:45 -0800,"London, ON'",Eastern Time (US & Canada),cond they put her on am elite tonite i think ...,"[2054, 8544, 6731, 4082, 5942, 663, 2984, 8678..."


In [223]:
test_df['negativereason'].fillna('NONE',inplace=True)

In [224]:
def test_runner(df):
    odf=df
    for i in range (len(df)):
        reason=df.loc[i].negativereason
        if reason != "NONE":
            
            odf.loc[i,"predicted"]="negative"
            print(reason, df.loc[i].predicted,odf.loc[i].predicted)
    return(odf)


In [225]:
df_out=test_runner(test_df)

Lost Luggage negative negative
Late Flight negative negative
Customer Service Issue negative negative
Customer Service Issue negative negative
Late Flight negative negative
Can't Tell negative negative
Can't Tell negative negative
Customer Service Issue negative negative
Late Flight negative negative
Can't Tell negative negative
Late Flight negative negative
Customer Service Issue negative negative
Cancelled Flight negative negative
Customer Service Issue negative negative
Customer Service Issue negative negative
Can't Tell negative negative
Flight Attendant Complaints negative negative
Customer Service Issue negative negative
Can't Tell negative negative
Cancelled Flight negative negative
Cancelled Flight negative negative
Customer Service Issue negative negative
Customer Service Issue negative negative
Customer Service Issue negative negative
Cancelled Flight negative negative
Customer Service Issue negative negative
Customer Service Issue negative negative
Late Flight negative negat

In [226]:
df_out.head()

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentence_code,predicted
0,570031016800161792,1.0000,Lost Luggage,1.0000,United,NaN,tpensari,NaN,0,@united well someone should tell that to the e...,NaN,2015-02-23 17:22:39 -0800,NaN,NaN,well someone should tell that to the employees...,"[9345, 7916, 7696, 8473, 8524, 8646, 8529, 302...",negative
1,569948646717812736,1.0000,NONE,NaN,American,NaN,Metalmikefisher,NaN,0,@AmericanAir Let's all have a extraordinary we...,NaN,2015-02-23 11:55:21 -0800,NaN,Eastern Time (US & Canada),let is all have a extraordinary week and make ...,"[4973, 4613, 627, 4010, 366, 3272, 9327, 703, ...",NaN
2,569469326547222528,0.6811,Late Flight,0.6811,United,NaN,jlongoria21,NaN,0,@united since I have an international connecti...,NaN,2015-02-22 04:10:42 -0800,"Rio Grande Valley, Texas",Central Time (US & Canada),since i have an international connection at am...,"[7753, 4273, 4010, 696, 4554, 2092, 929, 663, ...",negative
3,569512239209824256,1.0000,Customer Service Issue,0.6812,Delta,NaN,cshells4,NaN,0,@JetBlue woulda been nice of you to let us kno...,NaN,2015-02-22 07:01:13 -0800,Connecticut,NaN,woulda been nice of you to let us know that or...,"[9520, 1151, 5747, 5899, 9586, 8646, 4973, 907...",negative
4,567842370533867520,1.0000,Customer Service Issue,1.0000,US Airways,NaN,suzanneboles,NaN,0,@USAirways contd.. They put her on 7 pm flite ...,NaN,2015-02-17 16:25:45 -0800,"London, ON'",Eastern Time (US & Canada),cond they put her on am elite tonite i think ...,"[2054, 8544, 6731, 4082, 5942, 663, 2984, 8678...",negative


In [227]:
prediction_df=pd.DataFrame(test_df.predicted)

In [228]:
prediction_df.columns=["Sentiment"]

In [229]:
prediction_df.head()

,Sentiment
0,positive
1,positive
2,neutral
3,positive
4,positive


In [230]:
prediction_df.index.name="Index"

In [231]:
from datetime  import datetime

In [232]:
a=datetime.now().strftime("%Y%m%d_%H%M%S")

In [233]:
print(a)

20211011_194847


In [234]:
prediction_df.to_csv("submission_"+a+".csv")